In [2]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [3]:
df = pd.read_csv('Data/gold_futures_ohlcv.csv', parse_dates=['ts_event'])
df["ts_event"] = pd.to_datetime(df["ts_event"]).dt.tz_convert('America/Chicago') # ensure it's UTC
df = df.set_index('ts_event', inplace=False)

In [4]:
df.head(20)

,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol
ts_event,,,,,,,,,
2010-06-05 19:00:00-05:00,35,1,197012,1225.8,1226.3,1223.8,1224.0,23,GCZ0
2010-06-05 19:00:00-05:00,35,1,105130,-1.1,-1.0,-1.1,-1.0,24,GCN0-GCQ0
2010-06-05 19:00:00-05:00,35,1,197015,1232.0,1232.0,1230.0,1230.0,3,GCM1
2010-06-05 19:00:00-05:00,35,1,107004,-4.1,-4.1,-4.2,-4.2,20,GCQ0-GCZ0
2010-06-05 19:00:00-05:00,35,1,102338,1220.0,1221.4,1217.9,1218.0,27,GCN0
2010-06-05 19:00:00-05:00,35,1,104313,1221.1,1222.5,1218.3,1218.9,2031,GCQ0
2010-06-05 19:00:00-05:00,35,1,197009,1220.4,1220.7,1220.4,1220.7,2,GCM0
2010-06-05 19:00:00-05:00,35,1,145539,1220.5,1220.5,1220.5,1220.5,1,GCV0
2010-06-05 19:00:00-05:00,35,1,107006,-11.5,-11.5,-11.5,-11.5,3,GCQ0-GCM1


In [5]:
df[(df['symbol'] == "GCQ5") & (df.index.date > datetime.date(2023, 1, 1))].tail(10)
# 3325.8,3331.8,3263.9,3295.8,19252

,rtype,publisher_id,instrument_id,open,high,low,close,volume,symbol
ts_event,,,,,,,,,
2025-07-20 19:00:00-05:00,35,1,42008094,3355.3,3416.9,3353.8,3410.9,165116,GCQ5
2025-07-21 19:00:00-05:00,35,1,42008094,3411.1,3447.5,3394.9,3439.4,179361,GCQ5
2025-07-22 19:00:00-05:00,35,1,42008094,3439.3,3451.7,3390.6,3397.1,197465,GCQ5
2025-07-23 19:00:00-05:00,35,1,42008094,3397.2,3400.3,3355.2,3371.8,157037,GCQ5
2025-07-24 19:00:00-05:00,35,1,42008094,3371.7,3376.6,3325.5,3338.5,154916,GCQ5
2025-07-26 19:00:00-05:00,35,1,42008094,3321.1,3337.4,3303.0,3333.9,9902,GCQ5
2025-07-27 19:00:00-05:00,35,1,42008094,3334.0,3345.4,3300.0,3310.5,123454,GCQ5
2025-07-28 19:00:00-05:00,35,1,42008094,3310.3,3332.3,3305.8,3326.3,52866,GCQ5
2025-07-29 19:00:00-05:00,35,1,42008094,3326.1,3331.8,3263.9,3278.1,5261,GCQ5
